# Processing Big Data - Data Ingestion
© Explore Data Science Academy

## Honour Code
I {**PRAISE**, **TAIWO**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.



## Context 

To work constructively with any dataset, one needs to create an ingestion profile to make sure that the data at the source can be readily consumed. For this section of the predict, as the Data Engineer in the team, you will be required to design and implement the ingestion process. For the purposes of the project the AWS cloud storage service, namely, the S3 bucket service will act as your data source. All the data required can be found [here](https://processing-big-data-predict-stocks-data.s3.eu-west-1.amazonaws.com/stocks.zip).

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/DataIngestion.jpg"
     alt="Data Ingestion"
     style="float: center; padding-bottom=0.5em"
     width=40%/>
     <p><em>Figure 1. Data Ingestion</em></p>
</div>

Your manager, Gnissecorp Atadgib, knowing very well that you've recently completed your Data Engineering qualification, asks you to make use of Apache Spark for the ingestion as well as the rest of the project. His rationale being, that stock market data is generated every day and is quite time-sensitive and would require scalability when deploying to a production environment. 

## Dataset - US Nasdaq




<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/Nasdaq.png"
     alt="Nasdaq"
     style="float: center; padding-bottom=0.5em"
     width=50%/>
     <p><em>Figure 2. Nasdaq</em></p>
</div>

The data that you will be working with is a historical snapshot of market data taken from the Nasdaq electronic market. This dataset contains historical daily prices for all tickers currently trading on Nasdaq. The up-to-date list can be found on their [website](https://www.nasdaq.com/)


The provided data contains price data dating back from 02 January 1962 up until 01 April 2020. The data found in the S3 bucket has been stored in the following structure:

```
     stocks/<Year>/<Month>/<Day>/stocks.csv
```
Each CSV file for every trading day contains the following details:
- **Date** - specifies trading date
- **Open** - opening price
- **High** - maximum price during the day
- **Low** - minimum price during the day
- **Close** - close price adjusted for splits
- **Adj Close** - close price adjusted for both dividends and splits
- **Volume** - the number of shares that changed hands during a given day

## Basic initialisation
To get you started, let's import some basic Python libraries as well as Spark modules and functions.

In [1]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

Remember that we need a `SparkContext` and `SparkSession` to interface with Spark.
We will mostly be using the `SparkContext` to interact with RDDs and the `SparkSession` to interface with Python objects.

> ℹ️ **Instructions** ℹ️
>
>Initialise a new **Spark Context** and **Session** that you will use to interface with Spark.

In [2]:
#TODO: Write your code here
spark = SparkSession.builder.enableHiveSupport().getOrCreate()
spark

In [3]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Investigate dataset schema
At this point, it is enough to read in a single file to ascertain the data structure. You will be required to use the information obtained from the small subset to create a data schema. This data schema will be used when reading the entire dataset using Spark.

> ℹ️ **Instructions** ℹ️
>
>Make use of Pandas to read in a single file and investigate the plausible data types to be used when creating a Spark data schema. 
>
>*You may use as many coding cells as necessary.*

In [4]:

dirs = []
for path, subdirs, files in os.walk("../stocks/1962"):
    for name in files:
        dir = (os.path.join(path, name))
        dirs.append(dir)

files = []
for file in dirs:
    df = pd.read_csv(file)
    files.append(df)

df = pd.concat(files)
df.head(50)


,Date,Open,High,Low,Close,Adj Close,Volume,stock
0,1962-01-02,6.532155,6.556185,6.532155,6.532155,1.536658,55900.0,AA
1,1962-01-02,6.125844,6.160982,6.125844,6.125844,1.414651,59700.0,ARNC
2,1962-01-02,0.837449,0.837449,0.823045,0.823045,0.145748,352200.0,BA
3,1962-01-02,1.604167,1.619792,1.588542,1.604167,0.136957,163200.0,CAT
4,1962-01-02,0.000000,3.296131,3.244048,3.296131,0.051993,105600.0,CVX
5,1962-01-02,0.092908,0.096026,0.092908,0.092908,0.035517,817400.0,DIS
6,1962-01-02,0.000000,30.687500,30.375,30.375000,0.506943,1600.0,DTE
7,1962-01-02,0.000000,10.281250,10.125,10.125000,0.227351,25600.0,ED
8,1962-01-02,0.000000,7.687500,7.541667,7.583333,0.978945,49200.0,FL
9,1962-01-02,0.751202,0.763722,0.74369,0.748698,0.001782,2156500.0,GE


In [11]:
# # Import libraries
# import glob
# import pandas as pd

# # Get CSV files list from a folder
# path = "../stocks/1962"
# csv_files = glob.glob(path + "/*.csv")

# # Read each CSV file into DataFrame
# # This creates a list of dataframes
# df_list = (pd.read_csv(file) for file in csv_files)

# # Concatenate all DataFrames
# big_df   = pd.concat(df_list, ignore_index=True)

## Read CSV files

When working with big data, it is often not tenable to keep processing an entire data batch when you are in the process of development - this can be quite time-consuming. If the data is uniform, it is sufficient to work with a smaller subset to create basic functionality. Your manager has identified the year **1962** to perform the initial testing for data ingestion. 

> ℹ️ **Instructions** ℹ️
>
>Read in the data for **1962** using a data schema that purely uses string data types. You will be required to convert to the appropriate data types at a later stage.
>
>*You may use as many coding cells as necessary.*

In [12]:
#TODO: Write your code here
schema = StructType([StructField('date', StringType()),
                    StructField('open', StringType()),
                    StructField('high', StringType()),
                    StructField('low', StringType()),
                    StructField('close', StringType()),
                    StructField('adj_close', StringType()),
                    StructField('volume', StringType()),
                    StructField('stock', StringType())])

data = spark.createDataFrame(df, schema=schema)
data.show(50,False)

c:\Users\praise.taiwo\.conda\envs\pyspark-env\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+----------+------------------+------------------+------------------+------------------+--------------------+---------+-----+
|date      |open              |high              |low               |close             |adj_close           |volume   |stock|
+----------+------------------+------------------+------------------+------------------+--------------------+---------+-----+
|1962-01-02|6.5321550369262695|6.556184768676758 |6.5321550369262695|6.5321550369262695|1.5366575717926023  |55900.0  |AA   |
|1962-01-02|6.125843524932861 |6.160982131958008 |6.125843524932861 |6.125843524932861 |1.4146506786346436  |59700.0  |ARNC |
|1962-01-02|0.8374485373497009|0.8374485373497009|0.8230452537536621|0.8230452537536621|0.1457476019859314  |352200.0 |BA   |
|1962-01-02|1.6041666269302368|1.6197916269302368|1.5885416269302368|1.6041666269302368|0.1369574218988418  |163200.0 |CAT  |
|1962-01-02|0.0               |3.296130895614624 |3.2440476417541504|3.296130895614624 |0.0519925132393837  |105600.0 

## Update column names
To make the data easier to work with, you will need to make a few changes:
1. Column headers should all be in lowercase; and
2. Whitespaces should be replaced with underscores.


> ℹ️ **Instructions** ℹ️
>
>Make sure that the column headers are all in lowercase and that any whitespaces are replaced with underscores.
>
>*You may use as many coding cells as necessary.*

In [22]:
#TODO: Write your code here
data.show(20,False)

+----------+------------------+------------------+------------------+------------------+--------------------+---------+-----+
|date      |open              |high              |low               |close             |adj_close           |volume   |stock|
+----------+------------------+------------------+------------------+------------------+--------------------+---------+-----+
|1962-01-02|6.5321550369262695|6.556184768676758 |6.5321550369262695|6.5321550369262695|1.5366575717926023  |55900.0  |AA   |
|1962-01-02|6.125843524932861 |6.160982131958008 |6.125843524932861 |6.125843524932861 |1.4146506786346436  |59700.0  |ARNC |
|1962-01-02|0.8374485373497009|0.8374485373497009|0.8230452537536621|0.8230452537536621|0.1457476019859314  |352200.0 |BA   |
|1962-01-02|1.6041666269302368|1.6197916269302368|1.5885416269302368|1.6041666269302368|0.1369574218988418  |163200.0 |CAT  |
|1962-01-02|0.0               |3.296130895614624 |3.2440476417541504|3.296130895614624 |0.0519925132393837  |105600.0 

In [27]:
#TODO: Write your code here
data1 = spark.read.csv('../stocks/1962', header=True, recursiveFileLookup=True)
#TODO: Write your code here
for column in data1.columns:
    data1 = data1.withColumnRenamed(column, '_'.join(column.split()).lower())

## Null Values
Null values often represent missing pieces of data. It is always good to know where your null values lie - so you can quickly identify and remedy any issues stemming from these.

> ℹ️ **Instructions** ℹ️
>
>Write code to count the number of null values found in each column.
>
>*You may use as many coding cells as necessary.*

In [29]:
#TODO: Write your code here
from pyspark.sql.functions import *

data.select(*(count(when(col(c).isNull(), c)).alias(c) for c in data.columns)).show()


+----+----+----+---+-----+---------+------+-----+
|date|open|high|low|close|adj_close|volume|stock|
+----+----+----+---+-----+---------+------+-----+
|   0|   0|   0| 22|    0|        0|    21|    0|
+----+----+----+---+-----+---------+------+-----+



## Data type conversion - The final data schema

Now that we have identified the number of missing values in the data set, we'll move on to convert our data schema to the required data types. 

> ℹ️ **Instructions** ℹ️
>
>Use typecasting to convert the string data types in your current data schema to more appropriate data types.
>
>*You may use as many coding cells as necessary.*

In [30]:
#TODO: Write your code here

data2 = data1.withColumn("date",col("date").cast(DateType())) \
    .withColumn("open",col("open").cast(FloatType())) \
    .withColumn("high",col("high").cast(FloatType())) \
    .withColumn("low",col("low").cast(FloatType())) \
    .withColumn("close",col("close").cast(FloatType())) \
    .withColumn("adj_close",col("adj_close").cast(FloatType())) \
    .withColumn("volume",col("volume").cast(FloatType())) \
    .withColumn("stock",col("stock").cast(StringType()))
data2.printSchema()

root
 |-- date: date (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- adj_close: float (nullable = true)
 |-- volume: float (nullable = true)
 |-- stock: string (nullable = true)



In [25]:
data.show(20,False)

+----------+------------------+------------------+------------------+------------------+--------------------+---------+-----+
|date      |open              |high              |low               |close             |adj_close           |volume   |stock|
+----------+------------------+------------------+------------------+------------------+--------------------+---------+-----+
|1962-01-02|6.5321550369262695|6.556184768676758 |6.5321550369262695|6.5321550369262695|1.5366575717926023  |55900.0  |AA   |
|1962-01-02|6.125843524932861 |6.160982131958008 |6.125843524932861 |6.125843524932861 |1.4146506786346436  |59700.0  |ARNC |
|1962-01-02|0.8374485373497009|0.8374485373497009|0.8230452537536621|0.8230452537536621|0.1457476019859314  |352200.0 |BA   |
|1962-01-02|1.6041666269302368|1.6197916269302368|1.5885416269302368|1.6041666269302368|0.1369574218988418  |163200.0 |CAT  |
|1962-01-02|0.0               |3.296130895614624 |3.2440476417541504|3.296130895614624 |0.0519925132393837  |105600.0 

In [23]:
data2.show(20,False)

+----------+----------+-----------+----------+----------+------------+---------+-----+
|date      |open      |high       |low       |close     |adj_close   |volume   |stock|
+----------+----------+-----------+----------+----------+------------+---------+-----+
|1962-01-02|6.532155  |6.556185   |6.532155  |6.532155  |1.5366576   |55900.0  |AA   |
|1962-01-02|6.1258435 |6.160982   |6.1258435 |6.1258435 |1.4146507   |59700.0  |ARNC |
|1962-01-02|0.83744854|0.83744854 |0.82304525|0.82304525|0.1457476   |352200.0 |BA   |
|1962-01-02|1.6041666 |1.6197916  |1.5885416 |1.6041666 |0.13695742  |163200.0 |CAT  |
|1962-01-02|0.0       |3.296131   |3.2440476 |3.296131  |0.051992513 |105600.0 |CVX  |
|1962-01-02|0.09290839|0.096026115|0.09290839|0.09290839|0.035517246 |817400.0 |DIS  |
|1962-01-02|0.0       |30.6875    |30.375    |30.375    |0.5069433   |1600.0   |DTE  |
|1962-01-02|0.0       |10.28125   |10.125    |10.125    |0.22735077  |25600.0  |ED   |
|1962-01-02|0.0       |7.6875     |7.541666

## Consolidate missing values
We have to check if the data type conversion above was done correctly.
If the casting was not successful, a null value gets inserted into the dataframe. You can thus check for successful conversion by determining if any null values are included in the resulting dataframe.


> ℹ️ **Instructions** ℹ️
>
>Write code to compare the number of invalid entries (nulls) pre-conversion and post-conversion.
>
>*You may use as many coding cells as necessary.*

In [31]:
#TODO: Write your code here
data2.select(*(count(when(col(c).isNull(), c)).alias(c) for c in data2.columns)).show()

+----+----+----+---+-----+---------+------+-----+
|date|open|high|low|close|adj_close|volume|stock|
+----+----+----+---+-----+---------+------+-----+
|   0|   0|   0| 42|    0|       21|    21|    0|
+----+----+----+---+-----+---------+------+-----+



In [32]:
data2.summary().toPandas()

,summary,open,high,low,close,adj_close,volume,stock
0,count,5106,5106,5064,5106,5085,5085,5106
1,mean,1.0904873526012002,16.757624946793637,15.728619917198033,16.64199179044607,5.9866425135353065,540930.2458210423,None
2,stddev,2.364453525304909,53.914073481935446,51.322922898144,53.546771399008016,24.64637054715319,864596.244052551,None
3,min,0.0,0.0,0.052377883,0.053624976,4.0381454E-7,0.0,AA
4,25%,0.0,0.2454427,0.24348958,0.24739583,0.006592116,38400.0,None
5,50%,0.0,1.8046875,1.7851562,1.8007812,0.12449309,177600.0,None
6,75%,0.69547325,6.903353,6.7916665,6.8333335,0.7141595,734400.0,None
7,max,20.0,287.5,282.5,285.0,127.45936,1.95456E7,XOM


Here you should be able to see if any of your casts went wrong. 
Do not attempt to correct any missing values at this point. This will be dealt with in later sections of the predict.

## Generate parquet files
When writing in Spark, we typically use parquet format. This format allows parallel writing using Spark's optimisation while maintaining other useful things like metadata.

When writing, it is good to make sure that the data is sufficiently partitioned. 

Generally, data should be partitioned with one partition for every 200MB of data, but this also depends on the size of your cluster and executors. 


### Check the size of the dataframe before partitioning

In [33]:
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer

In [34]:
from functools import reduce
heights = [100, 2, 300, 10, 11, 1000]
max_height = reduce(lambda x, y: y if x < y else x, heights)
print(max_height)

1000


In [35]:
rdd = data2.rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
obj = rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)
size = sc._jvm.org.apache.spark.util.SizeEstimator.estimate(obj)
size_MB = size/1000000
partition = [int(size_MB/200), 2]
partitions = reduce(lambda x, y: y if x < y else x, partition)
# partitions = max([int(size_MB/200), 2])
print(f'The dataframe is {size_MB} MB. \nNumber of Partition is {partitions}')

The dataframe is 11.37204 MB. 
Number of Partition is 2


### Write parquet files to the local directory
> ℹ️ **Instructions** ℹ️
>
> Use the **coalesce** function and the number of **partitions** derived above to write parquet files to your local directory 
>
>*You may use as many coding cells as necessary.*

In [37]:
#TODO: Write your code here
data2.coalesce(2).write.format("parquet").save("Task_1")